In [1]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from utils import *
import PrintedSpikingNN_lP as pSNN
import PrintedSpikingNN_lP_New as New_pSNN
import pprint
import torch
from configuration import *
import time
start_time = time.time()

import wandb

from surrogate.RSNN import SpikeSynth

In [2]:
args = parser.parse_args([
    "--DATASET", "00",
    "--SEED", "0",
    "--projectname", "pLSNN",
    "--DEVICE", "cpu"
])

args = FormulateArgs(args)

print(f'Training network on device: {args.DEVICE}.')
MakeFolder(args)



train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

SetSeed(args.SEED)

setup = f"model_pSNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
print(f'Training setup: {setup}.')

msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

Training network on device: cpu.
{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 100,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
Training setup: model_pSNN_data_acuteinflammation_seed_00.model.


In [3]:
if os.path.isfile(f'{args.savepath}/{setup}'):
    print(f'{setup} exists, skip this training.')
    msglogger.info('Training was already finished.')
else:
    wandb.init(
        project="Spike-Synth-Full",
        name="Surrogate_GPT_wGradients", #setup + "_lighningwrapper",  # use your setup name as the run name
        config=vars(args)
    )
    
    topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
    wandb.config.update({"topology": topology})

    msglogger.info(f'Topology of the network: {topology}.')

    psnn = pSNN.PrintedSpikingNeuralNetwork(topology, args).to(args.DEVICE)
    #psnn = New_pSNN.LightningPrintedSpikingNetwork(topology, args, model_class=SpikeSynth, 
    #ckpt_path="surrogate/checkpoints-srnn/spike_model-epoch=92-val_loss=0.07.ckpt",
    #train_loader=train_loader,
    #valid_loader=valid_loader,
    #test_loader=test_loader).to(args.DEVICE)

    lossfunction = pSNN.LFLoss(args).to(args.DEVICE)
    optimizer = torch.optim.AdamW(psnn.GetParam(), lr=args.LR)
    #optimizer = torch.optim.Adam(psnn.network.GetParam(), lr=args.LR)

    if args.PROGRESSIVE:
        psnn, best = train_pnn_progressive(psnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
    else:
        psnn, best = train_pnn(psnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

    if best:
        if not os.path.exists(f'{args.savepath}/'):
            os.makedirs(f'{args.savepath}/')
        torch.save(psnn, f'{args.savepath}/{setup}')
        msglogger.info('Training if finished.')
        end_time = time.time()
        total_time = end_time - start_time
        msglogger.info(f'Total computation time: {total_time / 3600:.2f} hours ({total_time:.2f} seconds).')
    else:
        msglogger.warning('Time out, further training is necessary.')

    wandb.finish()

wandb: Currently logged in as: lupos to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Progressive Training:   0%|          | 0/10000000000 [00:00<?, ?it/s]

Epoch 0 [Train]:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0 [Valid]:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 [Train]:   0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 100 [Valid]:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 101 [Train]:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 101 [Valid]:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 102 [Train]:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 